In [29]:
import SimpleITK as sitk
import numpy as np
import os
import pandas as pd
import re

In [5]:
ResultList=[]
for root,folder,files in os.walk('/home2/nye/XY/ZH/RawFiles/Raw'):
    Files=list(filter(lambda x:'nii' in x,files))
    if Files!=[]:
        for f in Files:
            Result={}
            FileName=f.split('.')[0]
            PatientID=root.split('/')[-1]
            Path=os.path.join(root,f)
            Array=sitk.GetArrayFromImage(sitk.ReadImage(Path))
            ShapeString='_'.join(np.array(Array.shape).flatten().astype(str))
            I=int(Array.shape[0]/2)
            ArrayUse=Array[I,:,:]
            String='_'.join(ArrayUse.flatten().astype(str))
            Result={'FullArrayShape':ShapeString,
                    'SlicedArray_String':String,'SlicedArray_ID':I,
                    'PatientID':PatientID,'FileName':FileName,'NiiPath':Path}
            ResultList.append(Result)

In [30]:
InformationNeed=[["0008|0020",'StudyDate_0008_0020'],
                 ["0008|0050",'AccessionNumber_0008_0050'],
                 ["0008|0070",'Manufacturer_0008_0070'],
                 ["0008|103e",'SeriesDescription_0008_103e'],
                 ["0018|0023",'MRAcquisitionType_0018_0023'],
                 ["0018|0025",'AngioFlag_0018_0025'],["0018|1314",'FlipAngle_0018_1314'],
                 ["0018|0050",'SliceThickness_0018_0050'],
                 ["0018|0080",'RepetitionTime_0018_0080'],
                 ["0018|0081",'EchoTime_0018_0081'],
                 ["0018|0082",'InversionTime_0018_0082'],
                 ["0018|0084",'ImagingFrequency_0018_0084'],
                 ["0018|0087",'MagneticFieldStrength_0018_0087'],
                 ["0020|0011",'SeriesNumber_0020_0011'],
                 ["0010|0030",'DOB_0010_0030'],
                 ["0010|0040",'Gender_0010_0040'],['0010|0020','PatientID_0010_0040'],['0028|0010','Rows_0028_0010'],
                 ['0028|0011','Columns_0028_0011'],['0028|0030','PixelSpacing_0028_0030']]

In [50]:

ResultList=[]
CheckList=[]
for root,folder,files in os.walk('/Volumes/G-DRIVE Thunderbolt 3/ZH/RawDicom'):
    
    Files=list(filter(lambda x:'dcm' in x and 'Error' not in x, files))
    if Files!=[] and len(Files)>3:
        NeedCheck={}
        Result={}
        try:
            reader = sitk.ImageSeriesReader()
            series_found = reader.GetGDCMSeriesIDs(root)
            if len(series_found):
                ImageList=[]
                for serie in series_found:
                    dicom_names = reader.GetGDCMSeriesFileNames(root, serie)
                    reader.SetFileNames(dicom_names)
                    image = reader.Execute()
                    ImageList.append((image,dicom_names))
            for image,dicom_names in ImageList:
                Array=sitk.GetArrayFromImage(image)
                ShapeString='_'.join(np.array(Array.shape).flatten().astype(str))
                I=int(Array.shape[0]/2)
                ArrayUse=Array[I,:,:]
                String='_'.join(ArrayUse.flatten().astype('str'))
                Result.update({'FullArrayShape':ShapeString,
                               'SlicedArray_String':String,'SlicedArray_ID':I,
                               'DicomFileList':dicom_names,'DicomPathRoot':root})

                Dicom1=sitk.ReadImage(dicom_names[0])
                for HeadID,tag in InformationNeed:
                    try:
                        Result.update({tag:Dicom1.GetMetaData(HeadID)})
                    except:
                        pass
        
        except Exception as e:
            NeedCheck.update({'DicomPathRoot':root,'Error':e})
        CheckList.append(NeedCheck)
        ResultList.append(Result)
# DicomDf=pd.DataFrame(dict(enumerate(ResultList))).T        

In [51]:
NeedCheckDf=pd.DataFrame(dict(enumerate(list(filter(lambda x:x!={},CheckList))))).T

DicomDf=pd.DataFrame(dict(enumerate(list(filter(lambda x:x!={},ResultList))))).T

In [39]:
CheckList=[]
for i in NeedCheckDf.index:
    root=NeedCheckDf.DicomPathRoot[i]
    try:
        reader = sitk.ImageSeriesReader()
        series_found = reader.GetGDCMSeriesIDs(root)

        for serie in series_found:
            dicom_names = reader.GetGDCMSeriesFileNames(root, serie)
            reader.SetFileNames(dicom_names)
            image = reader.Execute()

        reader.SetFileNames(dicom_names)
    except Exception as e:
        NeedCheck.update({'DicomPathRoot':root,'Error':e})
    CheckList.append(NeedCheck)
pd.DataFrame(dict(enumerate(CheckList))).T.to_excel('E:/yingxiangzuxue/new/NeedCheckDf.xlsx')